In [4]:
import numpy as np
from scipy.stats import norm
import matplotlib.pyplot as plt


# Vectorized forward algorithm with scaling
def forward_algorithm(y, A, mu, sigma, pi):
    N = len(y)
    K = len(pi)
    
    B = norm.pdf(y[:, np.newaxis], mu, sigma)  # Emission probabilities

    alpha = np.zeros((N, K))
    c = np.zeros(N)
    
    # Initialization
    alpha[0, :] = B[0, :] * pi
    c[0] = np.sum(alpha[0, :])
    alpha[0, :] /= c[0]
    
    # Recursion (vectorized)
    for t in range(1, N):
        alpha[t, :] = B[t, :] * np.dot(alpha[t-1, :], A)
        c[t] = np.sum(alpha[t, :])
        alpha[t, :] /= c[t]
    
    return alpha, c, B

# Vectorized backward algorithm with scaling
def backward_algorithm(y, A, mu, sigma, c):
    N = len(y)
    K = A.shape[0]
    B = norm.pdf(y[:, np.newaxis], mu, sigma)  # Emission probabilities
    beta = np.zeros((N, K))
    
    # Initialization
    beta[N-1, :] = np.ones(K)
    
    # Recursion (vectorized)
    for t in range(N-2, -1, -1):
        beta[t, :] = np.dot(A, (B[t+1, :] * beta[t+1, :]))
        beta[t, :] /= c[t+1]
    
    return beta

# Vectorized computation of gamma and xi
def compute_gamma_xi(alpha, beta, A, B):
    N, K = alpha.shape
    gamma = alpha * beta
    
    xi = np.zeros((N-1, K, K))
    for t in range(N-1):
        xi[t, :, :] = np.outer(alpha[t, :], beta[t+1, :] * B[t+1, :]) * A
        xi[t, :, :] /= np.sum(xi[t, :, :])  # Normalize
    
    return gamma, xi

# Vectorized parameter update
def update_parameters(gamma, xi):

    # Update transition matrix A
    A_new = np.sum(xi, axis=0) / np.sum(gamma[:-1, :], axis=0)[:, np.newaxis]

    return A_new

# Vectorized Baum-Welch algorithm
def baum_welch(y, K, mu_init, sigma_init, max_iter=500, tol=1e-4):
    N = len(y)
    
    # Initialize parameters randomly
    A = np.ones((K, K)) / K

    # Initialize the parameters for the observation model with the given init
    mu = mu_init
    sigma = sigma_init
    pi = np.array([0.5, 0.5])
    
    log_likelihoods = []
    
    for iteration in range(max_iter):
        # print('iter = ', iteration+1)
        # E-step: Compute forward, backward, gamma, and xi
        alpha, c, B = forward_algorithm(y, A, mu, sigma, pi)
        beta = backward_algorithm(y, A, mu, sigma, c=c)
        gamma, xi = compute_gamma_xi(alpha, beta, A, B)
        
        # M-step: Update parameters
        A_new = update_parameters(gamma, xi)
        
        # Log-likelihood calculation
        log_likelihood = -np.sum(np.log(c))
        log_likelihoods.append(log_likelihood)
        
        # Check for convergence
        if iteration > 0 and abs(log_likelihoods[-1] - log_likelihoods[-2]) < tol:
            break
        # Update parameters
        A = A_new
    
    return A, gamma, log_likelihoods


# Function to generate the hidden states and observations
def generate_data(N, A, mu, sigma, pi):
    x = np.zeros(N, dtype=int)
    y = np.zeros(N)
    x[0] = np.random.choice(K, p=pi)
    y[0] = np.random.normal(mu[x[0]], sigma[x[0]])
    for t in range(1, N):
        x[t] = np.random.choice(K, p=A[x[t-1], :])
        y[t] = np.random.normal(mu[x[t]], sigma[x[t]])
    return x, y


# Parameters
N = 100  # Length of the sequence
K = 2  # Number of hidden states

p1, p2 = 0.9, 0.5  # iid sequence

EbN0_dB = 0
eta = 10**(EbN0_dB/10)
amp = np.sqrt(eta)

print('amp = ', amp)

# True transition probability matrix
A_true = np.array([[p1, 1-p1],
                   [1-p2, p2]])

# True emission parameters (means and standard deviations)
mu_true = np.array([-amp, amp])
sigma_true = np.array([1.0, 1.0])

# True initial state distribution
pi_true = np.array([0.5, 0.5])

# Generate the sequences
# np.random.seed(42)  # For reproducibility
x_seq, y_seq = generate_data(N, A_true, mu_true, sigma_true, pi_true)

# print('x_seq = ', x_seq)
# print('y_seq = ', y_seq)

# Run the Baum-Welch algorithm
A_est, gamma_est, log_likelihoods = baum_welch(y_seq, K, mu_init=mu_true, sigma_init=sigma_true)

print('gamma_est = ', gamma_est)

# Estimated state sequence
x_hat = np.argmax(gamma_est, axis=1)

print('A_est = ', A_est)

# Compare the estimated states with the true states
prob_err = np.sum(x_hat != x_seq) / N
print(f"Estimated States: {x_hat}")
print(f"True States:      {x_seq}")
print(f"Accuracy: {(1-prob_err)*100:.2f} %")


amp =  1.0
gamma_est =  [[8.76737480e-01 1.23262520e-01]
 [8.49235410e-01 1.50764590e-01]
 [9.41212025e-01 5.87879753e-02]
 [9.91119095e-01 8.88090456e-03]
 [9.94399568e-01 5.60043170e-03]
 [9.91357372e-01 8.64262796e-03]
 [9.56100706e-01 4.38992936e-02]
 [3.45167387e-03 9.96548326e-01]
 [7.14577094e-01 2.85422906e-01]
 [6.37491171e-01 3.62508829e-01]
 [7.39329482e-01 2.60670518e-01]
 [9.10194529e-01 8.98054715e-02]
 [8.63640497e-01 1.36359503e-01]
 [4.94527095e-01 5.05472905e-01]
 [9.54090158e-01 4.59098417e-02]
 [9.87775531e-01 1.22244690e-02]
 [9.44717946e-01 5.52820542e-02]
 [9.72760677e-01 2.72393227e-02]
 [3.51570979e-01 6.48429021e-01]
 [9.63031863e-01 3.69681365e-02]
 [9.72355600e-01 2.76444004e-02]
 [9.84703295e-01 1.52967051e-02]
 [9.92050841e-01 7.94915901e-03]
 [9.75532744e-01 2.44672561e-02]
 [9.57819170e-01 4.21808298e-02]
 [8.72841539e-01 1.27158461e-01]
 [9.96554538e-01 3.44546185e-03]
 [9.66062371e-01 3.39376292e-02]
 [9.37389451e-01 6.26105493e-02]
 [9.20117252e-01 7.

In [16]:
import numpy as np

# Vectorized forward algorithm with scaling
def forward_algorithm(y, A, O, pi):
    N = len(y)
    K = len(pi)
    
    # B = norm.pdf(y[:, np.newaxis], mu, sigma)  # Emission probabilities

    B = O[:, y].T

    alpha = np.zeros((N, K))
    c = np.zeros(N)
    
    # Initialization
    alpha[0, :] = B[0, :] * pi
    c[0] = np.sum(alpha[0, :])
    alpha[0, :] /= c[0]
    
    # Recursion (vectorized)
    for t in range(1, N):
        alpha[t, :] = B[t, :] * np.dot(alpha[t-1, :], A)
        c[t] = np.sum(alpha[t, :])
        alpha[t, :] /= c[t]
    
    return alpha, c, B


# Vectorized backward algorithm with scaling
def backward_algorithm(y, A, O, c):
    N = len(y)
    K = A.shape[0]

    B = O[:, y].T
    
    # B = norm.pdf(y[:, np.newaxis], mu, sigma)  # Emission probabilities
    beta = np.zeros((N, K))
    
    # Initialization
    beta[N-1, :] = np.ones(K)
    
    # Recursion (vectorized)
    for t in range(N-2, -1, -1):
        beta[t, :] = np.dot(A, (B[t+1, :] * beta[t+1, :]))
        beta[t, :] /= c[t+1]
    
    return beta

# Vectorized computation of gamma and xi
def compute_gamma_xi(alpha, beta, A, B):
    N, K = alpha.shape
    gamma = alpha * beta
    
    xi = np.zeros((N-1, K, K))
    for t in range(N-1):
        xi[t, :, :] = np.outer(alpha[t, :], beta[t+1, :] * B[t+1, :]) * A
        xi[t, :, :] /= np.sum(xi[t, :, :])  # Normalize
    
    return gamma, xi


A_true = np.array([[0.7,0.3],[0.3,0.7]])
O_true = np.array([[0.9,0.1],[0.2,0.8]])

y_seq = np.array([0,0,1,0,0])
pi_true = np.array([0.5,0.5])

alpha, c, B = forward_algorithm(y=y_seq, A=A_true, O=O_true, pi=pi_true)

beta = backward_algorithm(y=y_seq, A=A_true, O=O_true, c=c)

gamma, xi = compute_gamma_xi(alpha, beta, A_true, B)


print('alpha = ', alpha)

print('beta = ', beta)
print('gamma = ', gamma)








alpha =  [[0.81818182 0.18181818]
 [0.88335704 0.11664296]
 [0.19066794 0.80933206]
 [0.730794   0.269206  ]
 [0.86733889 0.13266111]]
beta =  [[1.06008087 0.72963611]
 [0.92875136 1.53957811]
 [1.61266533 0.85566414]
 [1.12264065 0.66707633]
 [1.         1.        ]]
gamma =  [[0.86733889 0.13266111]
 [0.82041905 0.17958095]
 [0.30748358 0.69251642]
 [0.82041905 0.17958095]
 [0.86733889 0.13266111]]
